# How to use `unfold`

Author: [Romain Sacchi](romain.sacchi@psi.ch)

Github repo: https://github.com/romainsacchi/

## fold
Several `brightway` databases are "folded" in a data package.

In [16]:
from unfold import Fold, Unfold
import numpy as np
import bw2data, bw2calc
import brightway2 as bw
bw2data.projects.set_current("new") # <-- name of the brightway project containing both the source database and the databases to fold

Consider a reference LCA database `reference database` and two other databases `db A` and `db B` which are slightly modified versions of `reference database`.

In [5]:
import yaml

fp = "reference_database.yaml"
with open(fp, "r") as stream:
    db = yaml.load(stream, Loader=yaml.FullLoader)
    
fp = "db A.yaml"

with open(fp, "r") as stream:
    db_a = yaml.load(stream, Loader=yaml.FullLoader)
    
fp = "db B.yaml"

with open(fp, "r") as stream:
    db_b = yaml.load(stream, Loader=yaml.FullLoader)
    
    
bw.Database("reference_database").write(db)
bw.Database("db A").write(db_a)
bw.Database("db B").write(db_b)

Writing activities to SQLite3 database:
0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/14/2023 17:32:33
  Finished: 03/14/2023 17:32:33
  Total time elapsed: 00:00:00
  CPU %: 108.10
  Memory %: 3.12


Writing activities to SQLite3 database:
0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/14/2023 17:32:34
  Finished: 03/14/2023 17:32:34
  Total time elapsed: 00:00:00
  CPU %: 106.60
  Memory %: 3.20


Writing activities to SQLite3 database:
0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/14/2023 17:32:34
  Finished: 03/14/2023 17:32:34
  Total time elapsed: 00:00:00
  CPU %: 111.30
  Memory %: 3.28


We calculate the supply array for 1 kg of `Activity A` from `db A` and `db B`.

In [6]:
lca = bw.LCA({bw.get_activity(("db A", "activity A")): 1})
lca.lci()
original_supply_A = lca.supply_array
original_supply_A

/opt/homebrew/Caskroom/miniforge/base/envs/unfold/lib/python3.10/site-packages/bw2calc/lca.py:228: UserWarning: No biosphere flows found. No inventory results can be calculated, `lcia` will raise an error
  warnings.warn("No biosphere flows found. No inventory results can "


array([ 1.        , 10.62500066, 19.56250163,  0.        ])

In [8]:
lca = bw.LCA({bw.get_activity(("db B", "activity A")): 1})
lca.lci()
original_supply_B = lca.supply_array
original_supply_B

array([ 1.        , 10.56250097, 19.1562522 ,  0.        ])

In [26]:
lca = bw.LCA({bw.get_activity(("reference_database", "activity A")): 1})
lca.lci()
reference = lca.supply_array

print(np.allclose(reference, original_supply_A) == False)
print(np.allclose(reference, original_supply_B) == False)
print(np.allclose(original_supply_A, original_supply_B) == False)

True
True
True


We can see that `db A` and `db B` are slightly different from one another, and also different from `reference database`.

Let's **fold** `db A` and `db B`.

In [11]:
Fold().fold(
    package_name="test",
    package_description="description of test",
    source="reference_database",
    system_model="cutoff",
    version="2.0",
    databases_to_fold=["db A", "db B"],
    descriptions=["this is db A", "this is db B"]
)

Extracting source database...
Done!
Extracting databases to fold...
Done!
Building scenario factor file...
Data package saved at test.zip


Now, let's **unfold** them, pointing at `reference database` as the reference database.

In [14]:
Unfold("test.zip").unfold(
    dependencies={
        "reference_database": "reference_database"
    }
)

The data package contains the following scenarios:
+-----+----------+--------------+
| No. | Scenario | Description  |
+-----+----------+--------------+
|  0  |   db A   | this is db A |
|  1  |   db B   | this is db B |
+-----+----------+--------------+

To unfold all scenarios, `unfold()`.
To unfold a specific scenario, `unfold(scenarios=[1,])`.
Getting activity data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 62368.83it/s]


Adding exchange data to activities


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 7936.24it/s]


Filling out exchange data


...

Extracting additional inventories...
Generating database for scenario db A...
Generating database for scenario db B...
Writing database for scenario db A...
Database db A already exists: it will be overwritten.



Writing activities to SQLite3 database:
0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/14/2023 17:37:09
  Finished: 03/14/2023 17:37:09
  Total time elapsed: 00:00:00
  CPU %: 102.20
  Memory %: 4.00
Created database: db A
Writing database for scenario db B...
Database db B already exists: it will be overwritten.


Writing activities to SQLite3 database:
0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/14/2023 17:37:10
  Finished: 03/14/2023 17:37:10
  Total time elapsed: 00:00:00
  CPU %: 100.60
  Memory %: 4.02
Created database: db B


Let's confirm that the supply arrays from 1 kg of `Activity A` from `db A` and `db B` are identical as before.

In [17]:
lca = bw.LCA({bw.get_activity(("db A", "activity A")): 1})
lca.lci()
new_supply_A = lca.supply_array
np.allclose(original_supply_A, new_supply_A)

True

In [18]:
lca = bw.LCA({bw.get_activity(("db B", "activity A")): 1})
lca.lci()
new_supply_B = lca.supply_array
np.allclose(original_supply_B, new_supply_B)

True

# unfold
## superstructure databases
`unfold` can also unfold [superstructure](https://link.springer.com/article/10.1007/s11367-021-01974-2) databases, to be used with [Activity Browser](https://github.com/LCA-ActivityBrowser/activity-browser).
It will write only one database, regardless of the number of scenarios.
It will also export a scenario difference spreadsheet to be read by Activity Browser.

In [22]:
Unfold("test.zip").unfold(
    dependencies={
        "reference_database": "reference_database"
    },
    superstructure=True # <-- NEW
)

The data package contains the following scenarios:
+-----+----------+--------------+
| No. | Scenario | Description  |
+-----+----------+--------------+
|  0  |   db A   | this is db A |
|  1  |   db B   | this is db B |
+-----+----------+--------------+

To unfold all scenarios, `unfold()`.
To unfold a specific scenario, `unfold(scenarios=[1,])`.
Getting activity data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 65793.00it/s]


Adding exchange data to activities


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 8290.78it/s]


Filling out exchange data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 90687.65it/s]

Extracting additional inventories...
Writing scenario difference file...
Generating superstructure database...
Scenario difference file exported to test.xlsx!

Writing superstructure database...



Writing activities to SQLite3 database:
0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/14/2023 17:39:59
  Finished: 03/14/2023 17:39:59
  Total time elapsed: 00:00:00
  CPU %: 108.20
  Memory %: 3.70
Created database: test


# unfold from an online repository

The repository [`replicate`](https://github.com/polca/replicate) lists data packages of scenario LCA databases.
We can download the data package to get REMIND scenario databases [here](https://zenodo.org/record/7476755).

And then we can unfold the scenarios of interest.

In [3]:
from unfold import Unfold
fp = "/Users/romain/Downloads/premise_remind_ssp2_base.zip"
u = Unfold(fp)

The data package contains the following scenarios:
+-----+---------------------------------+------------------------------------------------------------------------------+
| No. |             Scenario            |                                 Description                                  |
+-----+---------------------------------+------------------------------------------------------------------------------+
|  0  |    remind - SSP2-Base - 2005    |    Prospective db, based on REMIND, pathway SSP2-BASE, for the year 2005.    |
|  1  |    remind - SSP2-Base - 2010    |    Prospective db, based on REMIND, pathway SSP2-BASE, for the year 2010.    |
|  2  |    remind - SSP2-Base - 2015    |    Prospective db, based on REMIND, pathway SSP2-BASE, for the year 2015.    |
|  3  |    remind - SSP2-Base - 2020    |    Prospective db, based on REMIND, pathway SSP2-BASE, for the year 2020.    |
|  4  |    remind - SSP2-Base - 2025    |    Prospective db, based on REMIND, pathway SSP2-BASE, for t